In [44]:
from src.resnet import ResNet
import torch 
from torch import nn

In [45]:
tensor_a = torch.tensor([-10., 2., 3.])

torch.nn.functional.softplus(tensor_a)

tensor([4.5399e-05, 2.1269e+00, 3.0486e+00])

In [46]:
top_k = torch.topk(tensor_a, 1)
top_k

torch.return_types.topk(
values=tensor([3.]),
indices=tensor([2]))

In [47]:
tensor_a.masked_fill_(tensor_a != top_k.values, -float('inf'))

tensor([-inf, -inf, 3.])

In [48]:
class MoEGate(nn.Module):
    """
    Mixture of Experts (MoE) Gate module.
    """
    def __init__(self, input_channels: int, gate_dim: int, k: int, bias: bool = False) -> None:
        """
        Args:
            input_channels (int): Number of input channels.
            gate_dim (int): Dimensionality of the gate.
            k (int): Number of top gate values to keep.
            bias (bool, optional): Whether or not to add a bias term in the linear layers.
        """
        super().__init__()

        self.w_gate = nn.Linear(input_channels, gate_dim, bias=bias)
        self.w_noise = nn.Linear(input_channels, gate_dim, bias=bias)

        self.k = k
    
    def forward(self, x: torch.tensor) -> torch.tensor:
        """
        Computes the output of the MoE gate by computing the outputs of two linear layers,
        adding noise to one of them, and keeping only the top k largest values.

        Args:
            x (torch.tensor): Input tensor.

        Returns:
            torch.tensor: Output of the MoE gate.
        """
        print(f"x: {x.shape}")
        h_noise = self.w_noise(x)
        print(f"h_noise: {h_noise.shape}")
        noise = torch.randn_like(h_noise).to(x.device)
        h_noise = torch.nn.functional.softplus(h_noise).dot(noise)

        h_gate = self.w_gate(x)
        h = h_gate + h_noise

        top_k = torch.topk(h, k=self.k, dim=-1)
        output = h.masked_fill_(h != top_k.values, -float('inf'))

        return output

        

class MoE(nn.Module):
    def __init__(
        self,
        num_experts: int,
        input_channels: int,
        num_classes: int,
        channel_sizes: list[int],
        gate_dim: int,
        dropout: float,
        k: int
    ) -> None:
        """
        Dummy mixture of experts model.
        """
        super().__init__()
        self.num_experts = num_experts
        experts = []

        self.gate = MoEGate(input_channels=input_channels, gate_dim=gate_dim, k=k)

        for _ in range(num_experts):
            resnet = ResNet(input_channels, num_classes=num_classes, filters=channel_sizes, dropout=dropout)
            experts.append(resnet)
            del resnet.linear
            del resnet.flatten
            del resnet.avgpool
            del resnet.dropout
        
        self.experts = nn.ModuleList(experts)
        self.linear = nn.Linear(channel_sizes[-1], num_classes)
        self.dropout = nn.Dropout(p=dropout)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        
        
    
    def forward(self, x: torch.tensor):
        """
        The forward pass of the MoE model.
        """
        gate_out = self.gate(x)
        
        print(gate_out.shape)

In [49]:
moe = MoE(2, 3, 10, [8,16,32], 8, 0.1, 2)

In [50]:
moe

MoE(
  (gate): MoEGate(
    (w_gate): Linear(in_features=3, out_features=8, bias=False)
    (w_noise): Linear(in_features=3, out_features=8, bias=False)
  )
  (experts): ModuleList(
    (0-1): 2 x ResNet(
      (res_layers): Sequential(
        (0): ResBlock(
          (norm1): Sequential(
            (0): GroupNorm(1, 3, eps=1e-05, affine=True)
            (1): ReLU()
          )
          (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv2): Sequential(
            (0): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): GroupNorm(1, 8, eps=1e-05, affine=True)
          )
          (activation): ReLU()
          (idconv): Conv2d(3, 8, kernel_size=(1, 1), stride=(1, 1))
          (avgpool): Identity()
          (dropout): Dropout(p=0.2, inplace=False)
        )
        (1): ResBlock(
          (norm1): Sequential(
            (0): GroupNorm(1, 8, eps=1e-05, affine=True)
            (1): ReLU()
          )
          (co

In [51]:
moe(torch.randn(1, 3, 32, 32))

x: torch.Size([1, 3, 32, 32])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (96x32 and 3x8)